In [1]:
# !~/.local/bin/edge-tts --list-voices
# ms-MY-OsmanNeural, ms-MY-YasminNeural
# !~/.local/bin/edge-tts --voice ms-MY-OsmanNeural --text "saya suka makan ayam." --write-media test.wav

In [2]:
# !wget https://huggingface.co/datasets/huseinzol05/Malay-TTS-Yasmin/resolve/main/populated-parliament.json

In [3]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [4]:
import json

with open('populated-parliament.json') as fopen:
    data = json.load(fopen)

In [5]:
data[0]['text']

'PERDAGANGAN ANTARABANGSA DAN INDUSTRI menyatakan usaha memastikan pihak industri boleh mendapat para pekerja asing dengan kos munasabah tanpa dieksploitasikan dan birokrasi .'

In [6]:
import malaya

normalizer = malaya.normalize.normalizer()

/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.5.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(
/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/resource_loader.py:72: UserWarning: You are currently using TensorFlow 2.6.0 and trying to load a custom op (custom_ops/seq2seq/_beam_search_ops.so).
TensorFlow Addons has compiled its custom ops again

In [7]:
directory = 'osman-parliament-edge-tts'
directory_text = 'osman-parliament-edge-tts-text'

In [8]:
os.system(f'rm -rf {directory}')
os.system(f'rm -rf {directory_text}')

0

In [9]:
os.makedirs(directory, exist_ok = True)
os.makedirs(directory_text, exist_ok = True)

In [10]:
def generate_tts(text, filename):
    text = text.replace('"', r'\"').replace("'", r'\'')
    command = f'~/.local/bin/edge-tts --voice ms-MY-OsmanNeural --text "{text}" --write-media {filename}'
    # print(command)
    os.system(command)

In [11]:
data[7]

{'text': 'Daripada jumlah tersebut , sebanyak RM5.82 juta telah diagihkan kepada JPN Sabah untuk suku tahun pertama 2016 .',
 'cleaned': 'Daripada jumlah tersebut , sebanyak RM 5.82 juta telah diagihkan kepada JPN Sabah untuk suku tahun pertama 2016 .'}

In [12]:
n = normalizer.normalize('Daripada jumlah tersebut , / sebanyak RM5.82 bilion telah diagihkan kepada JPN Sabah untuk suku tahun pertama 2016 .',
                    normalize_entity = False)['normalize']

In [13]:
import re

def postprocessing(string):
    string = string.replace('/', ' ').replace('*', ' ').replace('=', ' ')
    return re.sub(r'[ ]+', ' ', string).strip()

postprocessing(n)

'Daripada jumlah tersebut , sebanyak lima bilion lapan ratus dua puluh juta ringgit telah diagihkan kepada JPN Sabah untuk suku tahun pertama dua ribu enam belas .'

In [14]:
n = normalizer.normalize('=== hidup === --', normalize_entity = False)['normalize']
n = postprocessing(n)
n

'hidup - -'

In [15]:
generate_tts(n, 'test.mp3')

In [16]:
import IPython.display as ipd
ipd.Audio('test.mp3')

In [17]:
from tqdm import tqdm

for i in tqdm(range(len(data))):
    try:
        
        filename = os.path.join(directory, f'{i}.mp3')
        filename_text = os.path.join(directory_text, f'{i}.json')
        
        if os.path.exists(filename_text):
            continue
            
        n = normalizer.normalize(data[i]['text'], normalize_entity = False)['normalize']
        n = postprocessing(n)
        if len(n):
            generate_tts(n, filename)
            with open(filename_text, 'w') as fopen:
                json.dump({'original': data[i]['text'], 'normalized': n}, fopen)
    except Exception as e:
        print(e)

  6%|████▉                                                                                   | 3336/59604 [56:55<15:45:36,  1.01s/it]

string index out of range


  9%|███████▌                                                                               | 5146/59604 [1:21:48<7:47:31,  1.94it/s]

string index out of range


 12%|██████████▌                                                                           | 7309/59604 [2:05:11<19:52:16,  1.37s/it]

...

 97%|████████████████████████████████████████████████████████████████████████████████████▏  | 57642/59604 [19:48:50<42:47,  1.31s/it]

string index out of range


 98%|█████████████████████████████████████████████████████████████████████████████████████▍ | 58564/59604 [20:10:11<22:28,  1.30s/it]

string index out of range


100%|███████████████████████████████████████████████████████████████████████████████████████| 59604/59604 [20:37:23<00:00,  1.25s/it]


In [18]:
# from pydub import AudioSegment
# sound = AudioSegment.from_mp3("test.mp3")
# sound.export("test.wav", format="wav")

In [20]:
from glob import glob
mp3s = glob(os.path.join(directory, '*.mp3'))
jsons = glob(os.path.join(directory_text, '*.json'))
len(mp3s)

59582

In [ ]:
import IPython.display as ipd
ipd.Audio('osman-parliament-edge-tts/12.mp3')

In [ ]:
with open('osman-parliament-edge-tts-text/15.txt') as fopen:
    print(fopen.read())

In [ ]:
for t in jsons:
    with open(t) as fopen:
        print(t, json.load(fopen))